<a href="https://colab.research.google.com/github/acmilannesta/Bert-embedding/blob/master/GoogleQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Tue Dec 10 13:31:02 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.36       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
!pip install keras-bert
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!unzip uncased_L-12_H-768_A-12.zip

  Created wheel for keras-bert: filename=keras_bert-0.80.0-cp36-none-any.whl size=37923 sha256=74505e1b8503dd9e11ba5ce26fde3b4ff699fcb31572e0bce462f04b38d442a4
  Stored in directory: /root/.cache/pip/wheels/63/dc/87/3260cb91f3aa32c0f85c5375429a30c8fd988bbb48f5ee21b0
  Created wheel for keras-transformer: filename=keras_transformer-0.31.0-cp36-none-any.whl size=13385 sha256=d0a06b0bb888f2e4494ae83632b7ee7e04ae46c494c901a1e0778286cd0ded05
  Stored in directory: /root/.cache/pip/wheels/a3/c5/9a/5a5130240be614a7a6fa786765d7692ae97f82601e2161bb56
  Created wheel for keras-pos-embd: filename=keras_pos_embd-0.11.0-cp36-none-any.whl size=7553 sha256=65dfcea0bad0702585d387d248515b2d0182f6ca1245031d0cad6855528787fc
  Stored in directory: /root/.cache/pip/wheels/5b/a1/a0/ce6b1d49ba1a9a76f592e70cf297b05c96bc9f418146761032
  Created wheel for keras-multi-head: filename=keras_multi_head-0.22.0-cp36-none-any.whl size=15371 sha256=079fdb0e05888d3fa4001b83811900b2d9553adf1497bfd542b39d4d5e06fc36
  Stor

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import numpy as np
import pandas as pd
import os, gc
import codecs
from keras.layers import *
from keras.callbacks import Callback
from keras.models import Model, load_model
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold, GroupKFold, KFold
from scipy.stats import spearmanr
from tqdm import tqdm
from keras_bert import load_trained_model_from_checkpoint, Tokenizer, AdamWarmup, calc_train_steps, get_custom_objects
from hyperopt import fmin, hp, tpe, STATUS_OK, Trials


Using TensorFlow backend.


In [0]:
data_path = 'drive/My Drive/GoogleQA/Data/'
train = pd.read_csv(data_path+'train.csv')
test = pd.read_csv(data_path+'test.csv')
sub = pd.read_csv(data_path+'sample_submission.csv')

# df_aux = pd.concat([train[['host', 'category']], test[['host', 'category']]], 0)
# aux1 = pd.get_dummies(df_aux['category'], drop_first=True)
# aux2 = pd.get_dummies(df_aux['host'].apply(lambda x: x.split('.')[0]), drop_first=True)
# aux = pd.concat([aux1, aux2], 1)
# train_aux = pd.concat([train, aux.iloc[:len(train), ]], 1)
# test_aux =  pd.concat([test, aux.iloc[len(train):, ]], 1)

##Chunknize answer

In [0]:
answer_col = [col for col in train.columns if col.startswith('answer') and
              col not in ['answer_user_name', 'answer_user_page']] + ['qa_id']
answer = train[answer_col]

answer_chunk = pd.DataFrame()
for row in tqdm(range(len(answer))):
    if len(answer.loc[row, 'answer']) % 256 != 0:
        replicate_row = len(answer.loc[row, 'answer']) // 256 + 1
    else:
        replicate_row = len(answer.loc[row, 'answer']) // 256
    for i in range(replicate_row):
        tmp = answer.loc[row].copy()
        tmp.loc['answer_chunk'] = tmp['answer'][i*256: (i+1)*256]
        answer_chunk = answer_chunk.append(tmp)

NameError: ignored

In [0]:
MAXLEN = 256 #@param {type:"slider", min:128, max:512, step:32}
BATCH_SIZE = 8 #@param {type:'slider', min:8, max:32, step:8}
NUM_EPOCHS = 3
NUM_CLASSES = 30
LR = 5e-5
MIN_LR = 0
# OUTPUT_TRAIN = 'train_bert_ipredcv1415_oof.csv'
# OUTPUT_TEST = 'test_bert_large.npy'
model_path = 'uncased_L-12_H-768_A-12' #@param ['uncased_L-12_H-768_A-12', 'wwm_uncased_L-24_H-1024_A-16', 'uncased_L-24_H-1024_A-16']


In [0]:
token_dict = {}
with codecs.open(os.path.join(model_path, 'vocab.txt'), 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)
tokenizer = Tokenizer(token_dict)

In [0]:
def convert_data(data_df, old_df, branch='training', type='answer'):
    data_df = data_df.reset_index(drop=True)
    global tokenizer
    global MAXLEN
    q_title, q_body, answer = [], [], [] 
    for i in tqdm(range(len(data_df))):
        if type=='answer':
            answer = []
            answer_ids, _ = tokenizer.encode(data_df.loc[i, 'answer_chunk'][:MAXLEN])
            answer.append(answer_ids)
        else:                       
            q_title_ids, _ = tokenizer.encode(data_df.loc[i, 'question_title'][:MAXLEN])
            q_title.append(q_title_ids)
            q_body_ids, _ = tokenizer.encode(data_df.loc[i, 'question_body'][:MAXLEN])
            q_body.append(q_body_ids)

    # aux = data_df[[col for col in data_df.columns if col not in old_df.columns]]
    if branch == 'training':
        targets = data_df.iloc[:, 11:41]
        return [q_title, q_body, answer], np.array(targets)
    else:
        return [q_title, q_body, answer]


In [0]:
"""## Data Generator"""
def seq_padding(X, padding=0):
    L = [len(x) for x in X]
    ML = max(L)
    return np.array([np.concatenate([x, [padding] * (ML - len(x))]) if len(x) < ML else x for x in X])

class data_generator:
    def __init__(self, data, batch_size=BATCH_SIZE, branch='train', q_a='a'):
        self.data = data
        self.batch_size = batch_size
        self.branch = branch
        self.q_a = q_a
        self.steps = len(self.data) // self.batch_size
        if len(self.data) % self.batch_size != 0:
            self.steps += 1

    def __len__(self):
        return self.steps

    def __iter__(self):
        while True:
            if self.branch == 'train':
                np.random.shuffle(self.data)
            for i in range(self.steps):
                d = self.data[i * self.batch_size: (i + 1) * self.batch_size]
                if q_a == 'q':
                    X1 = seq_padding([x[0] for x in d])
                    X2 = np.zeros_like(X1)
                    X3 = seq_padding([x[1] for x in d])
                    X4 = np.zeros_like(X3)
                    if self.branch == 'test':
                        # aux = np.array([x[3] for x in d])
                        yield [X1, X2, X3, X4]
                    else:
                        Y = np.array([x[2] for x in d])
                        # aux = np.array([x[4] for x in d])
                        yield [X1, X2, X3, X4], Y
                else:
                    X1 = seq_padding([x[0] for x in d])
                    X2 = np.zeros_like(X1)
                    if self.branch == 'test':
                        # aux = np.array([x[3] for x in d])
                        yield [X1, X2]
                    else:
                        Y = np.array([x[1] for x in d])
                        # aux = np.array([x[4] for x in d])
                        yield [X1, X2], Y
        


In [0]:
def model_build(len_train, q_a='a'):
    global NUM_CLASSES
    global BATCH_SIZE
    global NUM_EPOCHS
    global MIN_LR
    global LR
    global MAXLEN

    bert_model = load_trained_model_from_checkpoint(
        os.path.join(model_path, 'bert_config.json'),
        os.path.join(model_path, 'bert_model.ckpt'),
        seq_len = MAXLEN,
        trainable=True,
    )

    if q_a == 'q':
        q_in = Input(shape=(None,))
        q2_in = Input(shape=(None,))
        qb_in = Input(shape=(None,))
        qb2_in = Input(shape=(None,))

        q_inputs = bert_model([q_in, q2_in])
        q_outputs = Lambda(lambda x: x[:, 0])(q_inputs)
        qb_inputs = bert_model([qb_in, qb2_in])
        qb_outputs = Lambda(lambda x: x[:, 0])(qb_inputs)

        dense = concatenate([q_outputs, qb_outputs])
        outputs = Dense(NUM_CLASSES, activation='softmax')(dense)
        model = Model([q_in, q2_in, qb_in, qb2_in], outputs)
    else:
        a_in = Input(shape=(None,))
        a2_in = Input(shape=(None,))
        # aux_in = Input(shape=(aux.shape[1], ))

        a_inputs = bert_model([a_in, a2_in])
        a_outputs = Lambda(lambda x: x[:, 0])(a_inputs)

        dense = a_outputs
        outputs = Dense(NUM_CLASSES, activation='softmax')(dense)
        model = Model([a_in, a2_in], outputs)

    decay_steps, warmup_steps = calc_train_steps(
        len_train,
        batch_size=BATCH_SIZE,
        epochs=NUM_EPOCHS,
    )

    model.compile(
        loss='binary_crossentropy',
        # optimizer=Adam(1e-5),
        optimizer=AdamWarmup(
            decay_steps=decay_steps,
            warmup_steps=warmup_steps,
            lr=LR,
            min_lr=MIN_LR,
            ))
    del bert_model
    gc.collect()
    return model


In [0]:
test_x, test_aux = convert_data(test_aux, test, branch='testing')
pred = np.zeros((len(test), NUM_CLASSES))
kf = KFold(n_splits=5, shuffle=True, random_state=0)
idx = [x for x in kf.split(train_aux)]

def compute_spearmanr(trues, preds):
    rhos = []
    for col_trues, col_pred in zip(trues.T, preds.T):
        rhos.append(spearmanr(col_trues, col_pred + np.random.normal(0, 1e-7, col_pred.shape[0])).correlation)
    return np.nanmean(rhos)

class IntervalEval(Callback):
    def __init__(self, test_data, label):
        super(Callback, self).__init__()
        self.test_data = test_data
        self.label = label

    def on_epoch_end(self, epoch, logs={}):
        preds = self.model.predict_generator(self.test_data.__iter__(), len(self.test_data), verbose=1)
        score = compute_spearmanr(self.label, preds)
        print('Epoch - {:} - Spearman - {:.5f}'.format(epoch+1, score))

for i, (tr_idx, val_idx) in enumerate(idx[1:], 1):
    print('\nFold - {:}\n'.format(i))
    tr, val = train_aux.loc[tr_idx], train_aux.loc[val_idx]
    tr_x, tr_y, tr_aux = convert_data(tr, train)
    val_x, val_y, val_aux = convert_data(val, train)
    
    model = model_build(len_train=len(tr_aux))


    train_D = data_generator(list(zip(tr_x[0], tr_x[1], tr_x[2], tr_y, tr_aux)))
    valid_D = data_generator(list(zip(val_x[0], val_x[1], val_x[2], val_y, val_aux)), branch='valid')
    test_D = data_generator(list(zip(test_x[0], test_x[1], test_x[2], test_aux)), branch='test')
    ieval = IntervalEval(test_data=valid_D, label=val_y)
    model.fit_generator(
        train_D.__iter__(),
        steps_per_epoch=len(train_D),
        epochs=NUM_EPOCHS,
        callbacks = [ieval]
    )
    # oof_pred = model.predict_generator(valid_D.__iter__(), len(valid_D), verbose=1)
    # train_aug.loc[val_idx, 'oof_pred'] = np.argmax(oof_pred, 1)
    # print('oof - {:} f1_score - {:.4f}'.format(i, spearmanr(val_y, np.argmax(oof_pred, 1), average='weighted')))

    pred+= model.predict_generator(test_D.__iter__(), len(test_D), verbose=1) / kf.get_n_splits()
    # np.save(OUTPUT_TEST, pred)
    # s3.upload_file(Filename=OUTPUT_TEST, Bucket='acmilannesta', Key='large/'+OUTPUT_TEST)

    # model_file = 'model-oof-'+str(i)+'.h5'
    # model.save('model.h5')

    del model
    gc.collect()

  1%|          | 47/4863 [00:00<00:10, 461.73it/s]


Fold - 2



100%|██████████| 1216/1216 [00:02<00:00, 467.78it/s]


Epoch 1/3
152/152 [==============================] - 27s 181ms/step
Epoch - 1 - Spearman - 0.30811
Epoch 2/3
152/152 [==============================] - 19s 126ms/step
Epoch - 2 - Spearman - 0.32974
Epoch 3/3
152/152 [==============================] - 19s 126ms/step
Epoch - 3 - Spearman - 0.33309
60/60 [==============================] - 8s 131ms/step


  1%|          | 48/4863 [00:00<00:10, 475.20it/s]

...

  1%|          | 49/4864 [00:00<00:09, 485.13it/s]


Fold - 5



100%|██████████| 1215/1215 [00:02<00:00, 487.12it/s]


Epoch 1/3
 42/608 [=>............................] - ETA: 18:46 - loss: 1.6493

ResourceExhaustedError: ignored